In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.dataset as ds

from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline


# Loading the Data and observing

In [5]:
cols = [
    'Airline','Origin','Dest','CRSDepTime',
    'Month','DayOfWeek','Distance','DistanceGroup',
    'DepDel15','DepDelayMinutes',
    'ArrDel15','ArrDelayMinutes'
]

df = pd.read_parquet('./data/2021_delay_ds.parquet', columns=cols)

In [ ]:
data.shape

(4078318, 61)

In [6]:
df.shape


(6311871, 12)

In [ ]:
df.describe()

,CRSDepTime,Month,DayOfWeek,Distance,DistanceGroup,DepDel15,DepDelayMinutes,ArrDel15,ArrDelayMinutes
count,6.311871e+06,6.311871e+06,6.311871e+06,6.311871e+06,6.311871e+06,6.203458e+06,6.203458e+06,6.185870e+06,6.185870e+06
mean,1.323953e+03,6.969905e+00,4.013676e+00,7.955762e+02,3.653994e+00,1.731723e-01,1.276132e+01,1.726611e-01,1.252928e+01
std,4.741972e+02,3.300914e+00,2.006264e+00,5.832931e+02,2.281563e+00,3.783962e-01,4.736319e+01,3.779540e-01,4.674770e+01
min,1.000000e+00,1.000000e+00,1.000000e+00,3.100000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.200000e+02,4.000000e+00,2.000000e+00,3.700000e+02,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.315000e+03,7.000000e+00,4.000000e+00,6.460000e+02,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.723000e+03,1.000000e+01,6.000000e+00,1.033000e+03,5.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,6.000000e+00
max,2.359000e+03,1.200000e+01,7.000000e+00,5.812000e+03,1.100000e+01,1.000000e+00,3.095000e+03,1.000000e+00,3.089000e+03


In [8]:
df.isnull().sum().sort_values(ascending=False).head(20)

ArrDelayMinutes    126001
ArrDel15           126001
DepDelayMinutes    108413
DepDel15           108413
CRSDepTime              0
Dest                    0
Origin                  0
Airline                 0
DistanceGroup           0
Distance                0
DayOfWeek               0
Month                   0
dtype: int64

# Tasks 
- 4 predictions 
    - Classification
        1. ArrivalDelay -> Binary classification 
            - Will the flight reach the destination late ?
        2. DepartureDelay -> Binary Classification
            - Will the flight take off late ? 
    - Regression
        1. ArrivalDelay
        2. DepartureDelay


## TASK 1 : DepartureDelay (Binary Classification)
### Does the flight takes off late ?

In [9]:
# Remove rows where the target departure delay 15 is missing (null) as label is missing 
df_departed = df[df['DepDel15'].notna()].copy()

In [11]:
df_departed.shape

(6203458, 12)

In [15]:
df_departed.columns

Index(['Airline', 'Origin', 'Dest', 'CRSDepTime', 'Month', 'DayOfWeek',
       'Distance', 'DistanceGroup', 'DepDel15', 'DepDelayMinutes', 'ArrDel15',
       'ArrDelayMinutes'],
      dtype='str')

In [14]:
df_departed.info()

<class 'pandas.DataFrame'>
Index: 6203458 entries, 0 to 573778
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Airline          str    
 1   Origin           str    
 2   Dest             str    
 3   CRSDepTime       int64  
 4   Month            int64  
 5   DayOfWeek        int64  
 6   Distance         float64
 7   DistanceGroup    int64  
 8   DepDel15         float64
 9   DepDelayMinutes  float64
 10  ArrDel15         float64
 11  ArrDelayMinutes  float64
dtypes: float64(5), int64(4), str(3)
memory usage: 770.1 MB


In [25]:
df_departed[df_departed['DepDel15']==1].head()

,Airline,Origin,Dest,CRSDepTime,Month,DayOfWeek,Distance,DistanceGroup,DepDel15,DepDelayMinutes,ArrDel15,ArrDelayMinutes
36,SkyWest Airlines Inc.,FSM,DFW,617,3,3,227.0,1,1.0,22.0,0.0,14.0
51,SkyWest Airlines Inc.,DFW,FLG,2045,3,3,853.0,4,1.0,148.0,1.0,105.0
59,SkyWest Airlines Inc.,DFW,ASE,840,3,3,701.0,3,1.0,18.0,0.0,4.0
62,SkyWest Airlines Inc.,PHX,IAH,2005,3,3,1009.0,5,1.0,35.0,0.0,1.0
66,SkyWest Airlines Inc.,PHX,SLC,1843,3,3,507.0,3,1.0,109.0,1.0,89.0


In [26]:
features_departed_classification = [
'Airline', 'Origin', 'Dest', 'CRSDepTime', 'Month', 'DayOfWeek',
       'Distance', 'DistanceGroup', 'DepDel15', 'DepDelayMinutes'
]

x = df_departed[features_departed_classification]
y = df_departed['DepDel15']

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.2,random_state=8, stratify=y)

In [38]:
categorical_cols = ['Airline','Origin','Dest']

numeric_cols = [
    'CRSDepTime',
    'Month',
    'DayOfWeek',
    'Distance',
    'DistanceGroup'
]


In [39]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numeric_cols)
    ]
)


In [40]:
X_train_encoded = preprocessor.fit_transform(x_train)
X_test_encoded = preprocessor.transform(x_test)



MemoryError: Unable to allocate 151. MiB for an array with shape (39702128,) and data type int32

## TASK 2 : ArrivalDelay (Binary Classification) 
### Does the flight reaches destination late ?